In [1]:
from datetime import date
import warnings
warnings.filterwarnings('ignore')

In [2]:
from datetime import date

max_holding = 100
source = "CSV"

#dateTimeFormat = '%Y-%m-%d'
dateTimeFormat = '%d-%b-%Y'

# filter
days = 30 * 24

# frequency
freq = "1D"
symbol = 'BHARTIARTL'
myDate="31_Mar_2019"
filename ="./data/"+symbol+"_"+myDate+".csv"

# Calculate indicators and fields
%run "freedom_indicators_calculation.ipynb"


########## Algorithm #############
from enum import Enum
class S(Enum):
    BUY = 1
    SELL = 2
    HOLD = 3
    
class Trend(Enum):
    SIDE = 0
    UP = 1
    SUP = 2
    DOWN = 3
    SDOWN = 4

In [13]:
from talib import LINEARREG, BETA, LINEARREG_INTERCEPT, LINEARREG_SLOPE, STDDEV, TSF, ADOSC, VAR

# Math
price['priceSD'] = STDDEV(price.close)
price['priceVAR'] = VAR(price.close)

price['delBBUP'] = (price.bbup - price.close)/(price.bbup-price.bbmid)
price['delBBLOW'] = (price.close - price.bblow)/(price.bbmid-price.bblow)

price['d1Gradient'] = np.arctan(price.close - price.close.shift(1)) * 180 / np.pi
#price['2dGradient = np.arctan(price.close - price.close.shift(2)) * 180 / np.pi
price['d3Gradient'] = np.arctan(price.close - price.close.shift(3)) * 180 / np.pi
#price['4dGradient = np.arctan(price.close - price.close.shift(4)) * 180 / np.pi
price['d5Gradient'] = np.arctan(price.close - price.close.shift(5)) * 180 / np.pi
price['d10Gradient'] = np.arctan(price.close - price.close.shift(10)) * 180 / np.pi
price['d15Gradient'] = np.arctan(price.close - price.close.shift(15)) * 180 / np.pi

price['macd1dGradient'] = price.macd + 1 * (price.macd - price.macd.shift(1))


price['adosc'] = ADOSC(price.high, price.low, price.close, price.volume)

price['macdLR'] = LINEARREG(price.macd)
price['macdTSF'] = TSF(price.macd)

price['priceTSF'] = TSF(price.close)
price['priceLR'] = LINEARREG(price.close)


price['trend1D'] = pd.DataFrame(data = {"Trend":np.full(price.index.shape[0], Trend.SIDE)},index=price.index)
#price['trend1D'] = price[[price.d1Gradient >= 45]] = Trend.UP
#price['trend1D'] = price[[price.d1Gradient <= -45] = Trend.DOWN
#price.trend1D[price1dGradient >= 65] = Trend.SUP
#price.trend1D[price1dGradient <= -65] = Trend.SDOWN

price['trend3D'] = pd.DataFrame(data = {"Trend":np.full(price.index.shape[0], Trend.SIDE)},index=price.index)
#[price.d3Gradient >= 45] = Trend.UP
#price['trend3D[price.d3Gradient <= -45] = Trend.DOWN
#price.trend3D[price3dGradient >= 65] = Trend.SUP
#price.trend3D[price3dGradient <= -65] = Trend.SDOWN

price['trend5D'] = pd.DataFrame(data = {"Trend":np.full(price.index.shape[0], Trend.SIDE)},index=price.index)
#price['trend5D[price.d5Gradient >= 45] = Trend.UP
#price['trend5D[price.d5Gradient <= -45] = Trend.DOWN
#price.trend5D[price5dGradient >= 65] = Trend.SUP
#price.trend5D[price5dGradient <= -65] = Trend.SDOWN

price['trend10D'] = pd.DataFrame(data = {"Trend":np.full(price.index.shape[0], Trend.SIDE)},index=price.index)
#price['trend10D[price.d10Gradient >= 45] = Trend.UP
#price['trend10D[price.d10Gradient <= -45] = Trend.DOWN
#price.trend10D[price10dGradient >= 65] = Trend.SUP
#price.trend10D[price10dGradient <= -65] = Trend.SDOWN

price['trend15D'] = pd.DataFrame(data = {"Trend":np.full(price.index.shape[0], Trend.SIDE)},index=price.index)
#price['trend15D[price.d15Gradient >= 45] = Trend.UP
#price['trend15D[price.d15Gradient <= -45] = Trend.DOWN
#price.trend15D[price15dGradient >= 65] = Trend.SUP
#price.trend15D[price15dGradient <= -65] = Trend.SDOWN

price['chandlierLong'] = price.close-2*price.atr
price['chandlierShort'] = price.close+2*price.atr

price['sdHighLow'] = STDDEV((price.high - price.close))
price['varHighLow'] = STDDEV((price.high - price.close))
price['delPriceHighLow'] = (price.high - price.close)/price.sdHighLow


# Heikin-Ashi
price['HAopen'] = (price.open.shift(1) + price.close.shift(1))/2
price['HAhigh'] = price[['high','open','close']].max(axis = 1, skipna = True)
price['HAlow'] = price[['low','open','close']].min(axis = 1, skipna = True)
price['HAclose'] = (price.open+price.high+price.low+price.close)/4

In [14]:
price.dropna(inplace=True)
labels = pd.DataFrame(data = {"holdings":np.zeros(price.index.shape[0])},index=price.index)
#Entry Criterion
labels[
    (price.macd1dGradient > price.macdsignal)
] = S.BUY

#Exit Criterion
'''labels[
     
] = S.SELL
'''

# buy and sell decisions are stored in arrays for plotting purpose(visualisation)

buy = price[labels['holdings']==S.BUY]
sell = price[labels['holdings']==S.SELL]

In [15]:
#################################
#   Plotting Data              #
###############################

rangeMin = price.index[-150]
rangeMax = price.index[-1]

%run "freedom_plot.ipynb"

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x1,y2 ]
[ (3,1) x1,y3 ]
[ (4,1) x1,y4 ]
[ (5,1) x1,y5 ]
[ (6,1) x1,y6 ]
[ (7,1) x1,y7 ]
[ (8,1) x1,y8 ]
[ (9,1) x1,y9 ]



## Entry Criterion
- MACD crossover when market is trending up
- BB when market is moving sideways

## Exit Criterion
- Stochastics

## Identify market trend
- low ATR means market moving sideways
- Flat(gradient=0) MACD means market is moving sideways
- low ATR indicates low volatility - market is moving sideways
- High ATR indicates high volatility - market is trending
- Gradient of current - last 15 days